# Stream Write Notebook

This notebook performs the following operations:

- Read data from Apache Kafka Stream
- Write data to a Delta Lake table

In [ ]:
spark.stop()

In [10]:
# Special Apache Spark configuration with reduced resources for two applications

from pyspark.sql import SparkSession
import os

spark = SparkSession.builder \
    .appName("StreamWrite") \
    .master("spark://192.168.0.144:7077") \
    .config("spark.sql.extensions", "io.delta.sql.DeltaSparkSessionExtension") \
    .config("spark.sql.catalog.spark_catalog", "org.apache.spark.sql.delta.catalog.DeltaCatalog") \
    .config("spark.hadoop.hive.metastore.uris", "thrift://192.168.0.144:9083") \
    .config("spark.hadoop.javax.jdo.option.ConnectionURL", "jdbc:mysql://192.168.0.144:3306/metastore_db") \
    .config("spark.hadoop.javax.jdo.option.ConnectionDriverName", "com.mysql.cj.jdbc.Driver") \
    .config("spark.hadoop.javax.jdo.option.ConnectionUserName", "lh") \
    .config("spark.hadoop.javax.jdo.option.ConnectionPassword", os.getenv('MYSQL', 'Default_Value')) \
    .config("spark.jars", "/usr/local/spark/jars/delta-storage-3.2.0.jar,/usr/local/spark/jars/delta-spark_2.12-3.2.0.jar,/usr/local/spark/jars/kafka-clients-3.5.1.jar,/usr/local/spark/jars/spark-sql-kafka-0-10_2.12-3.5.1.jar, /usr/local/spark/jars/spark-token-provider-kafka-0-10_2.12-3.5.1.jar, /usr/local/spark/jars/commons-pool2-2.11.1.jar")    \
    .config("spark.delta.logStore.class", "org.apache.spark.sql.delta.storage.HDFSLogStore") \
    .config("spark.executor.memory", "4g") \
    .config("spark.executor.cores", "2") \
    .config("spark.driver.cores", "6") \
    .config("spark.driver.memory", "10g") \
    .config("spark.driver.maxResultSize", "2g") \
    .config("spark.hadoop.fs.defaultFS", "hdfs://192.168.0.144:9000") \
    .config("spark.databricks.delta.clusteredTable.enableClusteringTablePreview", "true") \
    .config("spark.sql.debug.maxToStringFields", "1000") \
    .config("spark.executor.instances", "2") \
    .config("spark.dynamicAllocation.enabled", "true") \
    .config("spark.dynamicAllocation.minExecutors", "1") \
    .config("spark.dynamicAllocation.maxExecutors", "2") \
    .enableHiveSupport() \
    .getOrCreate()

24/08/03 09:05:37 WARN Utils: Service 'SparkUI' could not bind on port 4040. Attempting port 4041.


In [ ]:
from pyspark.sql.functions import col, from_json
from pyspark.sql.types import StructType, StructField, StringType, IntegerType, LongType, DoubleType

kafka_bootstrap_servers = "192.168.0.145:9092"
kafka_topic = "delta"

schema = StructType([
    StructField("timestamp", StringType(), True),
    StructField("value", DoubleType(), True),
    StructField("country", StringType(), True),
    StructField("event_id", LongType(), True),
    StructField("actor_id", LongType(), True),
    StructField("year", IntegerType(), True),
    StructField("month", IntegerType(), True),
    StructField("day", IntegerType(), True),
    StructField("product_id", IntegerType(), True),
    StructField("location_id", IntegerType(), True),
    StructField("department_id", IntegerType(), True),
    StructField("campaign_id", IntegerType(), True),
    StructField("customer_id", IntegerType(), True),
])

kafka_stream = spark.readStream \
    .format("kafka") \
    .option("kafka.bootstrap.servers", kafka_bootstrap_servers) \
    .option("subscribe", kafka_topic) \
    .option("startingOffsets", "latest") \
    .load()

kafka_stream = kafka_stream.selectExpr("CAST(value AS STRING)")

df = kafka_stream.select(from_json(col("value"), schema).alias("data"))


df_flat = df.select(
    col("data.timestamp").cast("timestamp").alias("timestamp"),
    col("data.value").alias("value"),
    col("data.country").alias("country"),
    col("data.event_id").alias("event_id"),
    col("data.actor_id").alias("actor_id"),
    col("data.year").alias("year"),
    col("data.month").alias("month"),
    col("data.day").alias("day"),
    col("data.product_id").alias("product_id"),
    col("data.location_id").alias("location_id"),
    col("data.department_id").alias("department_id"),
    col("data.campaign_id").alias("campaign_id"),
    col("data.customer_id").alias("customer_id")
)

query = df_flat.writeStream \
    .format("delta") \
    .option("checkpointLocation", "/path/to/checkpoint/delta") \
    .option("mergeSchema", "true")  \
    .outputMode("append") \
    .start("/datalake/stream/kafka_delta_target")  

query.awaitTermination()


In [13]:
delta_table_path = "/datalake/stream/kafka_delta_target"
df = spark.read.format("delta").load(delta_table_path)

selected_columns_df = df.select(
    "value",
    "country",
    "event_id",
    "actor_id",
    "year",
    "month",
    "day",
    "product_id",
    "location_id",
    "department_id",
    "campaign_id",
    "customer_id"
)

selected_columns_df.printSchema()
selected_columns_df.show()


root
 |-- value: double (nullable = true)
 |-- country: string (nullable = true)
 |-- event_id: long (nullable = true)
 |-- actor_id: long (nullable = true)
 |-- year: integer (nullable = true)
 |-- month: integer (nullable = true)
 |-- day: integer (nullable = true)
 |-- product_id: integer (nullable = true)
 |-- location_id: integer (nullable = true)
 |-- department_id: integer (nullable = true)
 |-- campaign_id: integer (nullable = true)
 |-- customer_id: integer (nullable = true)


+--------------------+------------+--------+--------+----+------+--------+----------+-----------+-------------+-----------+-----------+
|               value|     country|event_id|actor_id|year| month|     day|product_id|location_id|department_id|campaign_id|customer_id|
+--------------------+------------+--------+--------+----+------+--------+----------+-----------+-------------+-----------+-----------+
|  0.9441410408078205|    Thailand|  176134|    4297|2024|202406|20240622|         1|          1|            1|          1|          1|
|   0.632545944339363|       Italy|  905906|    3525|2024|202406|20240622|         2|          2|            2|          2|          2|
|  0.6625826329665729|   Indonesia|  117763|    1957|2024|202406|20240622|         3|          3|            3|          3|          3|
|  0.8917914736796113|    Maldives|  686811|    7807|2024|202406|20240622|         4|          4|            4|          4|          4|
|  0.8537482285593957|       Egypt|  194755|    